# Modelo de tradução utilizando TED Talks
Traduzindo de inglês para português brasileiro, e vice-versa.

## Bibliotecas

In [1]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
nltk.download('punkt')

import torch
from torch import optim
from torch.utils.data import DataLoader

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install translate-toolkit

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from translate.storage.tmx import tmxfile

## Córpus
Vamos utilizar um córpus de legendas de TED talks

In [4]:
!wget https://object.pouta.csc.fi/OPUS-TED2020/v1/tmx/en-pt_br.tmx.gz
!gunzip en-pt_br.tmx.gz

--2022-03-17 15:36:44--  https://object.pouta.csc.fi/OPUS-TED2020/v1/tmx/en-pt_br.tmx.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32862474 (31M) [application/gzip]
Saving to: ‘en-pt_br.tmx.gz’

en-pt_br.tmx.gz     100%[===================>]  31.34M  12.7MB/s    in 2.5s    

2022-03-17 15:36:47 (12.7 MB/s) - ‘en-pt_br.tmx.gz’ saved [32862474/32862474]

gzip: en-pt_br.tmx already exists; do you wish to overwrite (y or n)? y


Lendo o córpus e separando em conjuntos de treino e teste

In [5]:
# ler córpus
with open("en-pt_br.tmx", 'rb') as fin:
    f_en2pt_br = tmxfile(fin, 'en', 'pt')
    f_pt_br2en = tmxfile(fin, 'pt', 'en')

prefixo_en2pt_br = '>>pt_br<<'
prefixo_pt_br2en = '>>en<<'

# formatar as traduções corretamente 
data_en2pt_br = [
                    { 'src': prefixo_en2pt_br + ' ' + w.source, 'trg': w.target } 
                    for w in f_en2pt_br.unit_iter()
                ]

data_pt_br2en = [
                    { 'src': prefixo_pt_br2en + ' ' + w.target, 'trg': w.source } 
                    for w in f_pt_br2en.unit_iter()
                ]

# separar em conjuntos de treino e teste
size_en2pt_br = int(len(data_en2pt_br) * 0.2)
treino_en2pt_br = data_en2pt_br[size_en2pt_br:][:10000]
teste_en2pt_br = data_en2pt_br[:size_en2pt_br][:1000]

size_pt_br2en = int(len(data_pt_br2en) * 0.2)
treino_pt_br2en = data_pt_br2en[size_pt_br2en:][:10000]
teste_pt_br2en = data_pt_br2en[:size_pt_br2en][:1000]

In [6]:
treino_en2pt_br[10]

{'src': ">>pt_br<< And as long as we've looked for explanations, we've wound up with something that gets closer and closer to science, which is hypotheses as to why we get sick, and as long as we've had hypotheses about why we get sick, we've tried to treat it as well. ",
 'trg': 'E à medida que procuramos explicações, vamos chegando a conclusões que se aproximam cada vez mais da ciência, que é a hipótese sobre porque adoecemos, e à medida que temos hipóteses sobre porque adoecemos, também procuramos nos tratar. '}

In [7]:
treino_pt_br2en[4234]

{'src': '>>en<< Essa é a verdade. ', 'trg': "That's the truth. "}

## Treinamento
Definindo parâmetros do modelo e treinamento

In [8]:
learning_rate = 1e-5 
epochs = 2
batch_size = 16
batch_status = 32
early_stop = 5
write_path_en2pt_br = 'model.pt'
write_path_pt_br2en = 'model.en'

Separando dados em batches ( lotes )

In [9]:
train_data_en2pt_br = DataLoader(treino_en2pt_br, batch_size = batch_size)
dev_data_en2pt_br = DataLoader(teste_en2pt_br, batch_size = batch_size)

train_data_pt_br2en = DataLoader(treino_pt_br2en, batch_size = batch_size)
dev_data_pt_br2en = DataLoader(treino_pt_br2en, batch_size = batch_size)

Método de avaliação

In [10]:
def evaluate(tokenizer, model, dev_data, batch_size, batch_status, device):

    model.eval()
    
    y_real = []
    y_pred = []
    
    for batch_idx, inp in enumerate(dev_data):
        y_real.extend(inp['trg'])
        
        # tokenização
        model_inputs = tokenizer(
            inp['src'], 
            truncation = True, 
            padding = True, 
            max_length = 128, 
            return_tensors = "pt"
        ).to(device)
        
        # tradução
        generated_ids = model.generate(**model_inputs, num_beams = 1)
        
        # pós-processamento da tradução
        output = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)
        y_pred.extend(output)
    
        # imprime resultados
        if (batch_idx+1) % batch_status == 0:
            print('Evaluation: [{}/{} ({:.0f}%)]'.format(batch_idx + 1, \
                len(dev_data), 100. * batch_idx / len(dev_data)))

    # cálculo BLUE score
    hyps, refs = [], []
    
    for i, snt_pred in enumerate(y_pred):
        hyps.append(nltk.word_tokenize(snt_pred))
        refs.append([nltk.word_tokenize(y_real[i])])
    
    bleu = corpus_bleu(refs, hyps)

    return bleu

Método do treinamento

In [11]:
def train(tokenizer, model, train_data, dev_data, optimizer, num_epochs, 
          batch_size, batch_status, device, write_path, early_stop = 5):
    
    max_bleu = evaluate(tokenizer, model, dev_data, batch_size, batch_status, device)
    print('BLEU inicial:', max_bleu)
    
    model.train()
    repeat = 0
    
    for epoch in range(num_epochs):
        losses = []
        batch_src, batch_trg = [], []

        for batch_idx, inp in enumerate(train_data):
            # inicializa zerando o gradiente
            optimizer.zero_grad()

            # tokenização
            model_inputs = tokenizer(
                inp['src'], 
                truncation = True,
                padding = True, 
                max_length = 128, 
                return_tensors = "pt"
            ).to(device)
            
            with tokenizer.as_target_tokenizer():
                labels = tokenizer(
                    inp['trg'], 
                    truncation = True, 
                    padding = True, 
                    max_length = 128, 
                    return_tensors = "pt"
                ).input_ids.to(device)
            
            # tradução
            output = model(**model_inputs, labels=labels) # forward pass

            # cálculo perda
            loss = output.loss
            losses.append(float(loss))

            # backpropagation
            loss.backward()
            optimizer.step()

            batch_src, batch_trg = [], []

            # imprime resultados
            if (batch_idx+1) % batch_status == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTotal Loss: {:.6f}'.format(
                epoch, batch_idx+1, len(train_data), 100. * batch_idx / len(train_data), 
                float(loss), round(sum(losses) / len(losses), 5)))

        bleu = evaluate(tokenizer, model, dev_data, batch_size, batch_status, device)
        print('BLEU:', bleu)
        
        if bleu > max_bleu:
            max_bleu = bleu
            repeat = 0

            print('Saving best model...')
            torch.save(model, write_path)
        else:
            repeat += 1

        if repeat == early_stop:
            break

Inicializando o modelo

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ROMANCE").to(device)
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ROMANCE")
optimizer = optim.AdamW(model.parameters(), lr = learning_rate)

Treinando inglês -> português brasileiro

In [14]:
train(
    tokenizer, model, train_data_en2pt_br, dev_data_en2pt_br, optimizer, epochs, 
    batch_size, batch_status, device, write_path_en2pt_br, early_stop
)

Evaluation: [32/63 (49%)]
BLEU inicial: 0.41589108912086625
Train Epoch: 0 [32/625 (5%)]	Loss: 0.571615	Total Loss: 0.919390
Train Epoch: 0 [64/625 (10%)]	Loss: 0.463009	Total Loss: 0.753600
Train Epoch: 0 [96/625 (15%)]	Loss: 0.820054	Total Loss: 0.680270
Train Epoch: 0 [128/625 (20%)]	Loss: 0.449106	Total Loss: 0.623230
Train Epoch: 0 [160/625 (25%)]	Loss: 0.503968	Total Loss: 0.598760
Train Epoch: 0 [192/625 (31%)]	Loss: 0.321125	Total Loss: 0.568030
Train Epoch: 0 [224/625 (36%)]	Loss: 0.433067	Total Loss: 0.554170
Train Epoch: 0 [256/625 (41%)]	Loss: 0.392383	Total Loss: 0.533040
Train Epoch: 0 [288/625 (46%)]	Loss: 0.893320	Total Loss: 0.539610
Train Epoch: 0 [320/625 (51%)]	Loss: 0.355073	Total Loss: 0.525290
Train Epoch: 0 [352/625 (56%)]	Loss: 0.444844	Total Loss: 0.515850
Train Epoch: 0 [384/625 (61%)]	Loss: 0.696781	Total Loss: 0.515320
Train Epoch: 0 [416/625 (66%)]	Loss: 0.392406	Total Loss: 0.505580
Train Epoch: 0 [448/625 (72%)]	Loss: 0.529203	Total Loss: 0.507200
Train 

Treinando português brasileiro -> inglês

In [15]:
train(
    tokenizer, model, train_data_pt_br2en, dev_data_pt_br2en, optimizer, epochs, 
    batch_size, batch_status, device, write_path_pt_br2en, early_stop
)

Evaluation: [32/625 (5%)]
Evaluation: [64/625 (10%)]
Evaluation: [96/625 (15%)]
Evaluation: [128/625 (20%)]
Evaluation: [160/625 (25%)]
Evaluation: [192/625 (31%)]
Evaluation: [224/625 (36%)]
Evaluation: [256/625 (41%)]
Evaluation: [288/625 (46%)]
Evaluation: [320/625 (51%)]
Evaluation: [352/625 (56%)]
Evaluation: [384/625 (61%)]
Evaluation: [416/625 (66%)]
Evaluation: [448/625 (72%)]
Evaluation: [480/625 (77%)]
Evaluation: [512/625 (82%)]
Evaluation: [544/625 (87%)]
Evaluation: [576/625 (92%)]
Evaluation: [608/625 (97%)]
BLEU inicial: 0.01053717427217039
Train Epoch: 0 [32/625 (5%)]	Loss: 1.290827	Total Loss: 1.680000
Train Epoch: 0 [64/625 (10%)]	Loss: 1.840085	Total Loss: 1.680880
Train Epoch: 0 [96/625 (15%)]	Loss: 1.909701	Total Loss: 1.639970
Train Epoch: 0 [128/625 (20%)]	Loss: 1.094319	Total Loss: 1.583320
Train Epoch: 0 [160/625 (25%)]	Loss: 1.285535	Total Loss: 1.537460
Train Epoch: 0 [192/625 (31%)]	Loss: 1.091980	Total Loss: 1.498820
Train Epoch: 0 [224/625 (36%)]	Loss: 1.0

## Resultados

Inglês -> Português brasileiro

In [16]:
# sentenças a serem traduzidas
batch_input_str = (
    (">>pt_br<< Please, don't fail me now."), 
    (">>pt_br<< Who is a good translator? You are!"), 
    (">>pt_br<< I hope you are able to translate a big sentence, because people nowadays love texting. And I want to present this to my teacher and colleagues, so you have to work!"),
    (">>pt_br<< I really don't want to study tonight but I have to do it because I want to graduate and get a job and have a lot of money.")
)

# tokenizando as sentenças
encoded = tokenizer(batch_input_str, return_tensors = 'pt', padding = True).to(device)

# traduzindo
translated = model.generate(**encoded)

# preparando a saída
tokenizer.batch_decode(translated, skip_special_tokens = True)

['Por favor, não falle comigo agora.',
 'Quem é um bom tradutor?',
 'Espero que você seja capaz de traduzir uma frase grande, porque as pessoas hoje adoram SMS. E eu quero apresentar isso ao meu professor e colegas, então você tem que trabalhar!',
 'Eu realmente não quero estudar hoje, mas eu tenho que fazer isso porque eu quero me formar, arrumar um emprego e ter muito dinheiro.']

Português brasileiro -> Inglês

In [17]:
batch_input_str = (
    (">>en<< Será que isso vai funcionar?"),
    (">>en<< Teste número 2. Você consegue traduzir isso que eu sei!"),
    (">>en<< Acho que eu preciso deixar você rodando por mais tempo, né?"),
    (">>en<< Eu sei que eu deveria ser mais criativo nos meus testes, mas não acredito que consegui traduzir de português brasileiro para inglês, mesmo com um BLEU tão baixo.")
)

# tokenizando as sentenças
encoded = tokenizer(batch_input_str, return_tensors = 'pt', padding = True).to(device)

# traduzindo
translated = model.generate(**encoded)

# preparando a saída
tokenizer.batch_decode(translated, skip_special_tokens = True)

['I will see what this will work?',
 "It's going to trade this that I know!",
 'I think I need to learn you walking for most time, yes?',
 "I know that I should be more creative in my trials, but I'm not believe that I can't trade brain to English brain translate with a BLEU's down."]